## 분류 모델 사용

## 데이터 수집

In [1]:
import pandas as pd
df_titanic = pd.read_csv('../../datasets/titanic_disaster_train.csv')
df_titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## 데이터분석

In [2]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
df_titanic['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

## 데이터 전처리

### 결측치 처리

In [4]:
df_titanic.dropna(inplace=True)

In [5]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


In [6]:
df_titanic['Survived'].value_counts()

Survived
1    123
0     60
Name: count, dtype: int64

### OnehotEncoding 처리

In [7]:
df_titanic['Pclass'].unique(), df_titanic['Pclass'].shape

(array([1, 3, 2]), (183,))

In [8]:
df_titanic[['Pclass']].shape

(183, 1)

In [9]:
from sklearn.preprocessing import OneHotEncoder

onehot_pclass = OneHotEncoder()
onehot_pclass.fit(df_titanic[['Pclass']])


OneHotEncoder()

In [10]:
onehot_pclass.categories_

[array([1, 2, 3])]

In [11]:
encoded_pclass = onehot_pclass.transform(df_titanic[['Pclass']]).toarray()
# encoded_pclass
encoded_pclass.shape, type(encoded_pclass)

((183, 3), numpy.ndarray)

In [12]:
# 원본 merge 위해 dataframe으로 변환
pclass_name_list = onehot_pclass.get_feature_names_out(['Pclass'])
# df_pclasses = pd.DataFrame(data=encoded_pclass
#                            , columns=pclass_name_list)
df_pclasses = pd.DataFrame(data=encoded_pclass
                           , columns=pclass_name_list
                           , index=df_titanic.index)
df_pclasses.head(2)

,Pclass_1,Pclass_2,Pclass_3
1,1.0,0.0,0.0
3,1.0,0.0,0.0


In [13]:
df_titanic_concated = pd.concat([df_titanic, df_pclasses]
                                , axis=1)
df_titanic_concated.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1.0,0.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1.0,0.0,0.0


### 스케일링 
- 지수화와 비슷

## 데이터 분할

In [17]:
features = df_titanic_concated.drop(columns=['Survived', 'PassengerId', 'Name'
                                    , 'Sex', 'Ticket', 'Cabin', 'Embarked'
                                    , 'Pclass'])
label = df_titanic_concated['Survived']
features.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

In [18]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       183 non-null    float64
 1   SibSp     183 non-null    int64  
 2   Parch     183 non-null    int64  
 3   Fare      183 non-null    float64
 4   Pclass_1  183 non-null    float64
 5   Pclass_2  183 non-null    float64
 6   Pclass_3  183 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 11.4 KB


In [19]:
# features['Pclass'].value_counts()

## 모델 학습

In [20]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression()
logisticRegression.fit(X=features, y=label)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
logisticRegression.coef_, logisticRegression.intercept_

(array([[-0.03927055,  0.17921947, -0.32220157,  0.00492597,  0.25214415,
          0.55950999, -0.55058711]]),
 array([1.64694663]))

## 모델 평가

In [22]:
from sklearn.metrics import accuracy_score      # 정확도

predicts = logisticRegression.predict(features)
predicts

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1])

In [23]:
accuracy_score(y_true=label, y_pred=predicts)

0.7049180327868853

In [24]:
predicts_1 = logisticRegression.predict(features[:1])
predicts_1

array([1])

In [25]:
logisticRegression.predict_proba(features[:1])

array([[0.28147255, 0.71852745]])

## 모델 배포